# Data preparation

In [1]:
import pandas as pd
import numpy as np

In [2]:
# Read the od_input data of employees traveling to other cities
df = pd.read_excel('../data/od_input_filtered.xlsx')
df.rename(columns={"Woonregio's": "origin", "Werkregio's": "destination", "Banen van werknemers (x 1 000)": "employees"}, inplace=True)
print('Number of rows: ',len(df))
# Take the mean of several years (2015-2019)
df= df.groupby(['origin', 'destination']).agg({'employees':['mean']})
df.columns = [col[0] for col in df.columns]
df.reset_index(inplace=True)

Number of rows:  6674


In [3]:
# Read in the municipality name data
municipalities = pd.read_excel('../data/Gemeenten alfabetisch 2020.xlsx')
municipalities = municipalities[municipalities['Provincienaam']=='Noord-Brabant']
# Municipality names
municipality_names = municipalities.Gemeentenaam.sort_values().unique()
print('Number of municipalities: ',len(municipality_names))
# Change municipality names to the correct 2019 ones.
"""
Aalburg + Werkendam + Woudrichem ------> Altena
Maasdonk ------------------------------> Den Bosch
Veghel, Schijndel, Sint-Oedenrode -----> Meierijstad
"""
df.replace({'Aalburg':'Altena', 'Werkendam':'Altena', 'Woudrichem':'Altena', 'Maasdonk':"'s-Hertogenbosch",
           'Veghel':'Meierijstad','Schijndel':'Meierijstad', 'Sint-Oedenrode':'Meierijstad'},inplace=True)

Number of municipalities:  62


In [4]:
# Adjust the number of travelers once the municipality names change
df= df.groupby(['origin', 'destination']).agg({'employees':['sum']})
df.columns = [col[0] for col in df.columns]
df.reset_index(inplace=True)

# Add missing origin-destination combinations

In [5]:
# Determine all missing origin-destination combinations
missing_destinations = 0
for i in municipality_names:
    missing_destinations += (62-len(df[df['origin']==i]))
print(missing_destinations)

2240


In [6]:
# Add the origin destination combinations that are missing
origin = []
destination = []
employees = []
for i in municipality_names:
    included = df[df['origin']==i]['destination'].unique()
    missing = list(set(municipality_names)-set(included))
    missing.sort()
    for j in missing:
        origin.append(i)
        destination.append(j)
        employees.append(0)
print(len(origin))

2240


In [7]:
# Add these origin-destination combinations
add_destinations = pd.DataFrame({'origin':origin, 'destination':destination, 'employees':employees})

In [8]:
# Merge these additions with the existing OD dataframe
df_merged = df.append(add_destinations, ignore_index=True)
df = df_merged

# Create OD matrix

In [9]:
# Create an OD matrix from the dataframe long format
df_pivot = (df.pivot(index='origin', columns='destination', values='employees'))
# Scale the number of travelers with 1000
df_pivot = df_pivot*1000
df_pivot.to_csv('../data/travelling_to_other_cities_x1000.csv')
# Create a numpy matrix of its values 
od_matrix = df_pivot.to_numpy()

# Create population size dataframe

In [10]:
# Load the inhabitant numbers per municipality
inh = pd.read_excel('../data/inwoners_per_gemeente_2019.xlsx')
inh.Municipality.replace({'s-Hertogenbosch':"'s-Hertogenbosch"},inplace=True)
# Sort according to the OD matrix dataframe
inh.sort_values(by='Municipality',inplace=True)
inh = inh.reset_index(drop=True)

# Add population size

In [11]:
# Sum of the OD matrix inhabitants that are traveling
travel_sum = od_matrix.sum(axis=1)
# Remaining part that is not traveling
remainers = inh.Inhabitants.values - travel_sum
# Add remainers to the OD matrix
np.fill_diagonal(od_matrix, od_matrix.diagonal() + remainers)
# Round to integer (remains a float though)
od_matrix = np.round(od_matrix, 0)
# Create a dataframe with the updated OD matrix values that now include the remainers
df_od_matrix = pd.DataFrame(od_matrix, columns=df_pivot.columns, index=df_pivot.index)
# Write to csv
df_od_matrix.to_csv('../data/df_od_matrix.csv')

# Checks for implementation

In [12]:
t = df_od_matrix.copy()
t

destination,'s-Hertogenbosch,Alphen-Chaam,Altena,Asten,Baarle-Nassau,Bergeijk,Bergen op Zoom,Bernheze,Best,Bladel,...,Steenbergen,Tilburg,Uden,Valkenswaard,Veldhoven,Vught,Waalre,Waalwijk,Woensdrecht,Zundert
origin,,,,,,,,,,,,,,,,,,,,,
's-Hertogenbosch,123832.0,100.0,340.0,0.0,0.0,100.0,150.0,820.0,420.0,120.0,...,0.0,3600.0,1240.0,100.0,400.0,1500.0,200.0,1280.0,500.0,100.0
Alphen-Chaam,100.0,6869.0,0.0,0.0,340.0,0.0,100.0,0.0,0.0,0.0,...,0.0,680.0,0.0,0.0,0.0,0.0,0.0,100.0,0.0,100.0
Altena,760.0,0.0,48426.0,0.0,0.0,0.0,100.0,0.0,0.0,0.0,...,0.0,560.0,0.0,0.0,0.0,0.0,0.0,1000.0,100.0,0.0
Asten,100.0,0.0,0.0,12150.0,0.0,0.0,0.0,0.0,100.0,0.0,...,0.0,160.0,100.0,0.0,100.0,0.0,0.0,0.0,0.0,0.0
Baarle-Nassau,100.0,140.0,0.0,0.0,5427.0,0.0,0.0,0.0,0.0,0.0,...,0.0,380.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Vught,2840.0,0.0,0.0,0.0,0.0,0.0,0.0,100.0,100.0,0.0,...,0.0,620.0,160.0,0.0,100.0,18879.0,0.0,100.0,100.0,100.0
Waalre,240.0,0.0,0.0,0.0,0.0,100.0,0.0,0.0,120.0,100.0,...,0.0,180.0,100.0,300.0,800.0,0.0,11857.0,0.0,0.0,0.0
Waalwijk,1480.0,0.0,300.0,0.0,100.0,0.0,100.0,100.0,100.0,0.0,...,0.0,2240.0,100.0,0.0,100.0,100.0,0.0,36000.0,300.0,100.0


In [13]:
inh.sort_values(by='Inhabitants',ascending=False)

,Municipality,Inhabitants
20,Eindhoven,231642
53,Tilburg,217259
13,Breda,183873
0,'s-Hertogenbosch,154205
31,Helmond,91524
...,...,...
48,Sint Anthonis,11606
38,Mill en Sint Hubert,10891
10,Boekel,10588
1,Alphen-Chaam,10149


In [14]:
# Total incoming travellers
t.sum(axis=0).sort_values(ascending=False)

destination
Eindhoven              268659.0
Tilburg                226979.0
Breda                  202861.0
's-Hertogenbosch       174665.0
Helmond                 89522.0
                         ...   
Sint Anthonis            9546.0
Mill en Sint Hubert      8904.0
Boekel                   8658.0
Alphen-Chaam             7949.0
Baarle-Nassau            6887.0
Length: 62, dtype: float64

In [15]:
# Total outgoing travellers (in my case its inhabitants????)
t.sum(axis=1).sort_values(ascending=False)

origin
Eindhoven              231642.0
Tilburg                217259.0
Breda                  183872.0
's-Hertogenbosch       154205.0
Helmond                 91524.0
                         ...   
Sint Anthonis           11606.0
Mill en Sint Hubert     10891.0
Boekel                  10588.0
Alphen-Chaam            10149.0
Baarle-Nassau            6847.0
Length: 62, dtype: float64

In [16]:
# Total incoming travellers
t.sum(axis=0)

destination
's-Hertogenbosch    174665.0
Alphen-Chaam          7949.0
Altena               50946.0
Asten                14890.0
Baarle-Nassau         6887.0
                      ...   
Vught                27067.0
Waalre               14077.0
Waalwijk             52040.0
Woensdrecht          23328.0
Zundert              19232.0
Length: 62, dtype: float64

In [17]:
# Total outgoing travellers (in my case its inhabitants????)
t.sum(axis=1)

origin
's-Hertogenbosch    154205.0
Alphen-Chaam         10149.0
Altena               55386.0
Asten                16710.0
Baarle-Nassau         6847.0
                      ...   
Vught                26396.0
Waalre               17247.0
Waalwijk             48240.0
Woensdrecht          21866.0
Zundert              21612.0
Length: 62, dtype: float64

In [18]:
t

destination,'s-Hertogenbosch,Alphen-Chaam,Altena,Asten,Baarle-Nassau,Bergeijk,Bergen op Zoom,Bernheze,Best,Bladel,...,Steenbergen,Tilburg,Uden,Valkenswaard,Veldhoven,Vught,Waalre,Waalwijk,Woensdrecht,Zundert
origin,,,,,,,,,,,,,,,,,,,,,
's-Hertogenbosch,123832.0,100.0,340.0,0.0,0.0,100.0,150.0,820.0,420.0,120.0,...,0.0,3600.0,1240.0,100.0,400.0,1500.0,200.0,1280.0,500.0,100.0
Alphen-Chaam,100.0,6869.0,0.0,0.0,340.0,0.0,100.0,0.0,0.0,0.0,...,0.0,680.0,0.0,0.0,0.0,0.0,0.0,100.0,0.0,100.0
Altena,760.0,0.0,48426.0,0.0,0.0,0.0,100.0,0.0,0.0,0.0,...,0.0,560.0,0.0,0.0,0.0,0.0,0.0,1000.0,100.0,0.0
Asten,100.0,0.0,0.0,12150.0,0.0,0.0,0.0,0.0,100.0,0.0,...,0.0,160.0,100.0,0.0,100.0,0.0,0.0,0.0,0.0,0.0
Baarle-Nassau,100.0,140.0,0.0,0.0,5427.0,0.0,0.0,0.0,0.0,0.0,...,0.0,380.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Vught,2840.0,0.0,0.0,0.0,0.0,0.0,0.0,100.0,100.0,0.0,...,0.0,620.0,160.0,0.0,100.0,18879.0,0.0,100.0,100.0,100.0
Waalre,240.0,0.0,0.0,0.0,0.0,100.0,0.0,0.0,120.0,100.0,...,0.0,180.0,100.0,300.0,800.0,0.0,11857.0,0.0,0.0,0.0
Waalwijk,1480.0,0.0,300.0,0.0,100.0,0.0,100.0,100.0,100.0,0.0,...,0.0,2240.0,100.0,0.0,100.0,100.0,0.0,36000.0,300.0,100.0


In [19]:
df

,origin,destination,employees
0,'s-Hertogenbosch,'s-Hertogenbosch,35.16
1,'s-Hertogenbosch,Alphen-Chaam,0.10
2,'s-Hertogenbosch,Altena,0.34
3,'s-Hertogenbosch,Bergeijk,0.10
4,'s-Hertogenbosch,Bergen op Zoom,0.15
...,...,...,...
3839,Zundert,Uden,0.00
3840,Zundert,Valkenswaard,0.00
3841,Zundert,Veldhoven,0.00
3842,Zundert,Vught,0.00


In [20]:
# This number is leaving Den Bosch
df[df['origin']=="'s-Hertogenbosch"].employees.sum()
# df[df['origin']=="Son en Breugel"].head(50)

65.53333333333333

In [21]:
# This number is entering Den Bosch
df[df['destination']=="'s-Hertogenbosch"].employees.sum()

85.99333333333334

In [22]:
# travel_sum = od_matrix.sum(axis=1)
# travel_sum

In [23]:
od_matrix.sum(axis=1)

array([154205.,  10149.,  55386.,  16710.,   6847.,  18491.,  66811.,
        30806.,  29821.,  20175.,  10588.,  29065.,  30747., 183872.,
        20440.,  24931.,  32362.,  26051.,  27150.,  19110., 231642.,
        43774.,  21515.,  39595.,  30447.,  26431.,  23793.,  12483.,
        14195.,  30194.,  15964.,  91524.,  44135.,  15334.,  22333.,
        15529.,  23327.,  80813.,  10891.,  36961.,  23186.,  18623.,
        26140.,  55616.,  91452.,  13060.,  77032.,  22572.,  11606.,
        28991.,  19322.,  16904.,  25054., 217259.,  41782.,  30910.,
        45337.,  26396.,  17247.,  48240.,  21866.,  21612.])